In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GroupKFold
from sklearn import linear_model 
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
df = pd.read_csv("data_vad.csv")
data = df[(df["Valence"].notna())&(df["Arousal"].notna())].drop("filename", axis = 1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3632 entries, 10 to 165128
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   msec      3632 non-null   int64  
 1   Corr      3632 non-null   float64
 2   Zyg       3632 non-null   float64
 3   Mas       3632 non-null   float64
 4   Valence   3632 non-null   float64
 5   Arousal   3632 non-null   float64
 6   label     3632 non-null   int64  
 7   batch_id  3632 non-null   object 
dtypes: float64(5), int64(2), object(1)
memory usage: 255.4+ KB


# Разделяем данные

In [3]:
X1 = data[data["label"] == 1]["batch_id"].unique()
X2 = data[data["label"] == 2]["batch_id"].unique()
X3 = data[data["label"] == 3]["batch_id"].unique()

In [4]:
X1_train, X1_test = train_test_split(X1,train_size = 0.7, random_state = 42)
X2_train, X2_test = train_test_split(X2,train_size = 0.7, random_state = 42)
X3_train, X3_test = train_test_split(X3,train_size = 0.7, random_state = 42)
X_train = [*X1_train, *X2_train, *X3_train]
X_test = [*X1_test, *X2_test, *X3_test]
data_train = data[data["batch_id"].apply(lambda x: x in X_train)]
data_test = data[data["batch_id"].apply(lambda x: x in X_test)]

In [5]:
X = data_train.iloc[:,1:4]
X_test = data_test.iloc[:,1:4]
y_av = data_train.iloc[:,4:6]
y_valence = data_train.iloc[:,4:5]
y_arousal = data_train.iloc[:,5:6]
y_valence_test = data_test.iloc[:,4:5]
y_arousal_test = data_test.iloc[:,5:6]
y_av_test = data_test.iloc[:,4:6]

# Модели до кросс-валидации

In [6]:
def models_test(X, y, X_test, y_test):
    model_lr = linear_model.LinearRegression()
    model_br = linear_model.BayesianRidge()
    model_en = linear_model.ElasticNet()
    model_svr = SVR()
    model_gbr = GradientBoostingRegressor()
    model_kne = KNeighborsRegressor()
    model_dtr = DecisionTreeRegressor()
    model_rfr = RandomForestRegressor()
    models = [model_lr, model_br, model_en, model_svr, model_gbr, model_kne, model_dtr, model_rfr]
    m = {"R2_train":{},"MAE_train":{},"MSE_train":{}, "R2_test":{},"MAE_test":{},"MSE_test":{}}
    for model in models:
        model.fit(X, y) 
        l = str(model)
        m["R2_train"][l[:l.index('(')]] = r2_score(y, model.predict(X))
        m["MAE_train"][l[:l.index('(')]] = mean_absolute_error(y, model.predict(X))
        m["MSE_train"][l[:l.index('(')]] = mean_squared_error(y, model.predict(X))
        m["R2_test"][l[:l.index('(')]] = r2_score(y_test, model.predict(X_test))
        m["MAE_test"][l[:l.index('(')]] = mean_absolute_error(y_test, model.predict(X_test))
        m["MSE_test"][l[:l.index('(')]] = mean_squared_error(y_test, model.predict(X_test))
    return pd.DataFrame(m)

In [7]:
TestModelsArousal = models_test(X, y_arousal, X_test, y_arousal_test)
TestModelsValence = models_test(X, y_valence, X_test, y_valence_test)

C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1891378733.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example u

In [8]:
TestModelsArousal

,R2_train,MAE_train,MSE_train,R2_test,MAE_test,MSE_test
LinearRegression,0.109401,0.073219,0.009967,0.001630,0.078125,0.008595
BayesianRidge,0.109398,0.073229,0.009967,0.001609,0.078150,0.008595
ElasticNet,0.061871,0.076248,0.010499,-0.018700,0.080689,0.008770
SVR,0.202261,0.074166,0.008928,-0.079891,0.081039,0.009297
GradientBoostingRegressor,0.427715,0.058242,0.006405,-0.025600,0.074611,0.008829
KNeighborsRegressor,0.481324,0.052917,0.005805,-0.131288,0.076039,0.009739
DecisionTreeRegressor,1.000000,0.000000,0.000000,-1.088246,0.095574,0.017977
RandomForestRegressor,0.897736,0.023258,0.001144,-0.165765,0.077571,0.010036


In [9]:
TestModelsValence

,R2_train,MAE_train,MSE_train,R2_test,MAE_test,MSE_test
LinearRegression,0.292631,0.245319,0.099994,0.031792,0.344960,0.177149
BayesianRidge,0.292628,0.245287,0.099994,0.032476,0.344764,0.177024
ElasticNet,0.280653,0.247715,0.101687,0.055822,0.337354,0.172752
SVR,0.339913,0.225432,0.093310,0.028515,0.333476,0.177749
GradientBoostingRegressor,0.528358,0.198331,0.066671,0.158884,0.314551,0.153896
KNeighborsRegressor,0.596347,0.176434,0.057061,0.057931,0.323869,0.172367
DecisionTreeRegressor,1.000000,0.000000,0.000000,-0.344659,0.361698,0.246027
RandomForestRegressor,0.921560,0.076556,0.011088,0.126230,0.311708,0.159870


# Подбор гиперпараметров с помощью кросс-валидации

#### Доля отложной и обучающейся выборки на каждой итерации

In [10]:
def contents_fold(cv, data):
    X1 = data[data["label"] == 1]["batch_id"].unique()
    X2 = data[data["label"] == 2]["batch_id"].unique()
    X3 = data[data["label"] == 3]["batch_id"].unique()
    a1 = np.array_split(X1, cv)
    a2 = np.array_split(X2, cv)
    a3 = np.array_split(X3, cv)
    n = data.shape[0]
    d = {'train':[], 'test':[]}
    for i in range(cv):
        X_test = [*a1[i], *a2[i], *a3[i]] 
        X_train = np.concatenate((*a1[i + 1:], *a2[i + 1:], *a3[i + 1:], *a1[:i], *a2[:i], *a3[:i]), axis = 0)
        data_train = data[data["batch_id"].apply(lambda x: x in X_train)]
        data_test = data[data["batch_id"].apply(lambda x: x in X_test)]
        d['train'].append(data_train.shape[0]/n)
        d['test'].append(data_test.shape[0]/n)
    return d

#### Доработанная под эту задачу KFold

In [11]:
def my_KFold(model, cv, data, name):
    k = 4
    if name == 'Arousal':
        k = 5
    arr_r2_train = []
    arr_r2_test = []
    arr_mae_train = []
    arr_mae_test = []
    arr_mse_train = []
    arr_mse_test = []
    #Разделение с учетом экспериментов
    X1 = data[data["label"] == 1]["batch_id"].unique()
    X2 = data[data["label"] == 2]["batch_id"].unique()
    X3 = data[data["label"] == 3]["batch_id"].unique()
    a1 = np.array_split(X1, cv)
    a2 = np.array_split(X2, cv)
    a3 = np.array_split(X3, cv)
    for i in range(cv):
        X_deffer = [*a1[i], *a2[i], *a3[i]] 
        X_train = np.concatenate((*a1[i + 1:], *a2[i + 1:], *a3[i + 1:], *a1[:i], *a2[:i], *a3[:i]), axis = 0)
        data_train = data[data["batch_id"].apply(lambda x: x in X_train)]
        data_test = data[data["batch_id"].apply(lambda x: x in X_deffer)]
        X = data_train.iloc[:,1:4]
        X_test = data_test.iloc[:,1:4]
        y = data_train.iloc[:,k:k + 1]
        y_test = data_test.iloc[:,k:k + 1]
        #Обучение фолда
        model.fit(X, y)
        #Оценка качества на тренировочной и отложенной выборках
        arr_r2_train.append(r2_score(y, model.predict(X)))
        arr_r2_test.append(r2_score(y_test, model.predict(X_test)))
        arr_mae_train.append(mean_absolute_error(y, model.predict(X)))
        arr_mae_test.append(mean_absolute_error(y_test, model.predict(X_test)))
        arr_mse_train.append(mean_squared_error(y, model.predict(X)))
        arr_mse_test.append(mean_squared_error(y_test, model.predict(X_test)))
    d = {'r2_train':np.mean(np.array(arr_r2_train)),
         'r2_test':np.mean(np.array(arr_r2_test)),
         'mae_train':np.mean(np.array(arr_mae_train)),
         'mae_test':np.mean(np.array(arr_mae_test)),
         'mse_train':np.mean(np.array(arr_mse_train)),
         'mse_test':np.mean(np.array(arr_mse_test))}
    return d

# DecisionTreeRegressor подбор гиперпараметрa max_depth

In [15]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(1,12):
    d = my_KFold(DecisionTreeRegressor(max_depth = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)

In [16]:
#Для Arousal
pd.DataFrame(d_nw)

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.116491,0.073673,0.009874,-0.016336,0.077485,0.011053,1
1,0.168459,0.071707,0.009287,-0.002097,0.075918,0.010641,2
2,0.224189,0.068718,0.008666,-0.057943,0.077185,0.011281,3
3,0.287576,0.065047,0.007965,-0.069920,0.076523,0.011394,4
4,0.350637,0.060945,0.007252,-0.092891,0.075040,0.011450,5
5,0.416550,0.056237,0.006517,-0.172344,0.076355,0.012143,6
6,0.481884,0.051779,0.005789,-0.253584,0.078533,0.012906,7
7,0.551196,0.046884,0.005016,-0.358418,0.081848,0.013876,8
8,0.624389,0.041227,0.004198,-0.449911,0.083773,0.014682,9
9,0.693365,0.035725,0.003431,-0.543544,0.086366,0.015623,10


In [17]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(1,12):
    d = my_KFold(DecisionTreeRegressor(max_depth = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)

In [18]:
#Для Valence
pd.DataFrame(d_nw)

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.227356,0.250473,0.109021,0.169651,0.255302,0.112634,1
1,0.283999,0.246040,0.101012,0.200828,0.252783,0.106875,2
2,0.340735,0.235165,0.093002,0.151047,0.258874,0.109816,3
3,0.424647,0.218289,0.081108,0.251651,0.247950,0.101097,4
4,0.478622,0.205599,0.073543,0.237110,0.247248,0.101212,5
5,0.527881,0.192316,0.066670,0.153184,0.254928,0.111674,6
6,0.581887,0.176813,0.059086,0.072828,0.259541,0.120017,7
7,0.630702,0.162373,0.052165,0.010608,0.263259,0.123515,8
8,0.687420,0.144047,0.044151,-0.051552,0.270238,0.132200,9
9,0.739758,0.125848,0.036778,-0.094578,0.274752,0.137403,10


# KNeighborsRegressor подбор гиперпараметрa n_neighbors

##### Для Valence

In [19]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(1,201, 10):
    d = my_KFold(KNeighborsRegressor(n_neighbors = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,1.000000,0.000000,0.000000,-0.466612,0.310450,0.174674,1
1,0.532516,0.193457,0.065941,0.236620,0.244701,0.098510,11
2,0.492711,0.205326,0.071575,0.275895,0.241541,0.095198,21
3,0.477787,0.210091,0.073676,0.286896,0.240766,0.094231,31
4,0.464168,0.213605,0.075596,0.292095,0.240086,0.093920,41
5,0.451883,0.216303,0.077341,0.294979,0.240033,0.093783,51
6,0.441529,0.218714,0.078795,0.298343,0.240037,0.093700,61
7,0.433052,0.220784,0.079999,0.296542,0.240824,0.094088,71
8,0.425150,0.222703,0.081106,0.294755,0.241421,0.094530,81
9,0.418403,0.224486,0.082054,0.294299,0.241706,0.094694,91


In [20]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(51,71):
    d = my_KFold(KNeighborsRegressor(n_neighbors = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.451883,0.216303,0.077341,0.294979,0.240033,0.093783,51
1,0.450720,0.216515,0.077504,0.294464,0.240165,0.093856,52
2,0.449580,0.216777,0.077664,0.294865,0.240124,0.093895,53
3,0.448618,0.217027,0.077796,0.294607,0.240237,0.094000,54
4,0.447290,0.217325,0.077981,0.295476,0.240128,0.093931,55
5,0.446229,0.217581,0.078130,0.296866,0.240006,0.093777,56
6,0.445157,0.217836,0.078281,0.297866,0.239920,0.093652,57
7,0.444414,0.217989,0.078382,0.297559,0.240060,0.093713,58
8,0.443236,0.218235,0.078551,0.297160,0.240152,0.093731,59
9,0.442713,0.218465,0.078626,0.297241,0.240166,0.093755,60


###### Для Arousal

In [21]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(1,201, 10):
    d = my_KFold(KNeighborsRegressor(n_neighbors = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,1.000000,0.000000,0.000000,-0.794848,0.094703,0.018365,1
1,0.390445,0.058416,0.006807,-0.031935,0.074639,0.010887,11
2,0.330585,0.061782,0.007478,0.023572,0.072757,0.010413,21
3,0.310541,0.062817,0.007702,0.055850,0.071970,0.010158,31
4,0.296034,0.063568,0.007867,0.070444,0.071788,0.010054,41
5,0.280625,0.064349,0.008042,0.078261,0.071858,0.010015,51
6,0.267778,0.065093,0.008186,0.082021,0.071968,0.010013,61
7,0.257719,0.065669,0.008300,0.087679,0.072089,0.009975,71
8,0.248374,0.066257,0.008406,0.093509,0.072176,0.009948,81
9,0.240090,0.066799,0.008500,0.096148,0.072289,0.009942,91


In [22]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(91,111):
    d = my_KFold(KNeighborsRegressor(n_neighbors = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.240090,0.066799,0.008500,0.096148,0.072289,0.009942,91
1,0.239383,0.066841,0.008508,0.095998,0.072296,0.009943,92
2,0.238687,0.066895,0.008516,0.096639,0.072289,0.009941,93
3,0.237757,0.066948,0.008527,0.097257,0.072294,0.009936,94
4,0.237310,0.066977,0.008532,0.097268,0.072292,0.009937,95
5,0.236556,0.067031,0.008541,0.096879,0.072342,0.009944,96
6,0.235802,0.067074,0.008550,0.097043,0.072370,0.009946,97
7,0.234979,0.067118,0.008559,0.097299,0.072410,0.009946,98
8,0.234407,0.067155,0.008565,0.096897,0.072455,0.009951,99
9,0.233642,0.067197,0.008574,0.097395,0.072475,0.009948,100


# RandomForestRegressor 

###### для Valence

In [23]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(1,11, 2):
    d = my_KFold(RandomForestRegressor(max_depth = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.240610,0.247558,0.107194,0.189944,0.251016,0.109517,1
1,0.382367,0.227309,0.087047,0.265952,0.241726,0.098005,3
2,0.509381,0.201655,0.069265,0.298562,0.236970,0.094453,5
3,0.613122,0.177200,0.054628,0.266933,0.238902,0.097691,7
4,0.717642,0.150850,0.039857,0.238613,0.241003,0.100169,9


In [24]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(3, 7):
    d = my_KFold(RandomForestRegressor(max_depth = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.383548,0.226779,0.086869,0.270142,0.241594,0.097890,3
1,0.453189,0.213126,0.077164,0.307246,0.236984,0.093529,4
2,0.510535,0.202003,0.069116,0.300273,0.236862,0.094185,5
3,0.560442,0.189879,0.062067,0.284099,0.238095,0.095785,6


In [34]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in [10, 50, 100, 200, 300, 400, 500]:
    d = my_KFold(RandomForestRegressor(max_depth = 4, n_estimators = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.446974,0.213833,0.078084,0.295972,0.239427,0.094754,10
1,0.452340,0.212725,0.077295,0.307768,0.236047,0.093009,50
2,0.454074,0.212915,0.077056,0.309495,0.236186,0.093433,100
3,0.454178,0.212724,0.077042,0.309767,0.236072,0.093375,200
4,0.455391,0.212688,0.076871,0.307828,0.236840,0.093559,300
5,0.454886,0.212849,0.076936,0.310827,0.236042,0.093087,400
6,0.454597,0.212896,0.076976,0.308876,0.236369,0.093258,500


In [42]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(1, 7):
    d = my_KFold(RandomForestRegressor(max_depth = 4, n_estimators = 200, min_samples_leaf = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.454308,0.212670,0.077031,0.306030,0.236710,0.093456,1
1,0.453201,0.212831,0.077173,0.307732,0.236591,0.093472,2
2,0.454431,0.212715,0.076985,0.308987,0.236416,0.093319,3
3,0.454246,0.213066,0.077024,0.307683,0.236881,0.093433,4
4,0.451534,0.213300,0.077400,0.306738,0.236975,0.093568,5
5,0.452342,0.213436,0.077284,0.309490,0.236116,0.093186,6


In [45]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(2, 6):
    d = my_KFold(RandomForestRegressor(max_depth = 4, n_estimators = 200, min_samples_leaf = 3, min_samples_split = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.454304,0.212856,0.077025,0.309182,0.236339,0.093051,2
1,0.453376,0.212872,0.077151,0.307758,0.236624,0.093267,3
2,0.453533,0.212836,0.077113,0.307447,0.236625,0.093549,4
3,0.453626,0.212956,0.077107,0.310422,0.236347,0.093301,5


In [47]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(2, 10):
    d = my_KFold(RandomForestRegressor(max_depth = 4, n_estimators = 200, min_samples_leaf = 3, max_features = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.446154,0.215846,0.078150,0.296636,0.238818,0.094743,2
1,0.454356,0.212725,0.077014,0.305835,0.236544,0.093491,3
2,0.453679,0.212924,0.077105,0.306093,0.236627,0.093548,4
3,0.453733,0.212707,0.077106,0.307086,0.236888,0.093497,5
4,0.452971,0.213051,0.077188,0.309519,0.236297,0.093221,6
5,0.454929,0.212854,0.076932,0.310852,0.235871,0.093067,7
6,0.454135,0.212755,0.077047,0.306935,0.236710,0.093465,8
7,0.454108,0.212615,0.077049,0.304954,0.237242,0.093747,9


In [48]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(2, 10):
    d = my_KFold(RandomForestRegressor(max_depth = 4, n_estimators = 200, min_samples_leaf = 3, max_features = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.445568,0.215615,0.078243,0.295325,0.238957,0.095124,2
1,0.453223,0.212867,0.077168,0.306882,0.236191,0.093403,3
2,0.454681,0.212903,0.076966,0.308295,0.236458,0.093314,4
3,0.452615,0.212888,0.077251,0.308208,0.236283,0.093390,5
4,0.453582,0.212888,0.077125,0.309341,0.236160,0.093236,6
5,0.454497,0.213108,0.076974,0.311396,0.236200,0.092995,7
6,0.453654,0.212949,0.077115,0.310679,0.236333,0.093098,8
7,0.453865,0.212594,0.077096,0.305861,0.236708,0.093561,9


In [76]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(1, 9):
    d = my_KFold(RandomForestRegressor(max_depth = 4, n_estimators = 200, min_samples_leaf = 3, max_features = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.393722,0.225817,0.085558,0.267142,0.244789,0.099429,1
1,0.443601,0.215865,0.078487,0.296386,0.238659,0.095085,2
2,0.453381,0.213082,0.077144,0.308092,0.236356,0.093478,3
3,0.453870,0.212773,0.077087,0.306466,0.236692,0.093771,4
4,0.454805,0.212684,0.076950,0.309552,0.236239,0.093307,5
5,0.453338,0.212681,0.077146,0.309786,0.235726,0.093189,6
6,0.455628,0.212835,0.076833,0.308832,0.236227,0.093334,7
7,0.453235,0.212918,0.077169,0.307633,0.236890,0.093597,8


In [54]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(2, 9):
    d = my_KFold(RandomForestRegressor(max_depth = 4, n_estimators = 200, min_samples_leaf = 3, max_features = 7, min_samples_split = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.452361,0.212759,0.077295,0.304550,0.237103,0.093762,2
1,0.453832,0.212594,0.077101,0.309279,0.236002,0.093173,3
2,0.453493,0.212890,0.077129,0.306342,0.236933,0.093692,4
3,0.453622,0.212738,0.077116,0.307883,0.236456,0.093290,5
4,0.453826,0.212647,0.077083,0.308349,0.236147,0.093321,6
5,0.454091,0.212741,0.077059,0.305976,0.236614,0.093575,7
6,0.453730,0.212925,0.077109,0.307257,0.236453,0.093563,8


In [55]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(2, 9):
    d = my_KFold(RandomForestRegressor(max_depth = 4, n_estimators = 200, min_samples_leaf = 3, max_features = 7, min_samples_split = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.453851,0.213015,0.077071,0.309398,0.236235,0.093243,2
1,0.455015,0.212821,0.076929,0.308058,0.236293,0.093394,3
2,0.455023,0.212721,0.076925,0.309579,0.236096,0.093073,4
3,0.453117,0.212882,0.077172,0.308581,0.236104,0.093251,5
4,0.453803,0.212926,0.077098,0.307441,0.236805,0.093437,6
5,0.453882,0.212888,0.077078,0.308305,0.236509,0.093295,7
6,0.453538,0.212903,0.077122,0.307030,0.236499,0.093517,8


In [56]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(2, 9):
    d = my_KFold(RandomForestRegressor(max_depth = 4, n_estimators = 200, min_samples_leaf = 3, max_features = 7, min_samples_split = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.453777,0.212936,0.077087,0.308369,0.236600,0.093383,2
1,0.454084,0.213016,0.077053,0.307538,0.236735,0.093529,3
2,0.453946,0.212719,0.077084,0.308120,0.236304,0.093387,4
3,0.453261,0.213052,0.077155,0.307331,0.236676,0.093583,5
4,0.453004,0.213017,0.077181,0.310390,0.235814,0.093177,6
5,0.454279,0.212903,0.077028,0.306424,0.236405,0.093496,7
6,0.452675,0.212756,0.077253,0.305968,0.236623,0.093455,8


###### Для Arousal

In [59]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(1, 10, 1):
    d = my_KFold(RandomForestRegressor(max_depth = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.135494,0.072829,0.009656,0.008007,0.076845,0.010857,1
1,0.199271,0.070479,0.008943,0.026370,0.075068,0.010465,2
2,0.267933,0.067319,0.008184,0.003977,0.075445,0.010659,3
3,0.337364,0.063230,0.007406,0.007054,0.073758,0.010553,4
4,0.406568,0.059318,0.006634,-0.001948,0.073066,0.010577,5
5,0.474836,0.055588,0.005872,-0.008592,0.072740,0.010653,6
6,0.542722,0.051637,0.005114,-0.033976,0.073321,0.010896,7
7,0.610243,0.047670,0.004356,-0.049346,0.073704,0.011053,8
8,0.676633,0.043412,0.003614,-0.069818,0.074385,0.011234,9


In [73]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in [10, 50, 100, 150, 200, 300, 400, 500]:
    d = my_KFold(RandomForestRegressor(max_depth = 2, n_estimators = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.193788,0.070816,0.009003,0.009419,0.075460,0.010660,10
1,0.199824,0.070424,0.008937,0.028571,0.075056,0.010460,50
2,0.199445,0.070438,0.008942,0.028361,0.075110,0.010456,100
3,0.200569,0.070443,0.008930,0.027527,0.075074,0.010462,150
4,0.199783,0.070464,0.008938,0.031450,0.074965,0.010429,200
5,0.200764,0.070432,0.008928,0.027710,0.075094,0.010476,300
6,0.200454,0.070417,0.008931,0.028854,0.075004,0.010437,400
7,0.200122,0.070447,0.008935,0.028286,0.075030,0.010453,500


In [82]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(1, 9):
    d = my_KFold(RandomForestRegressor(max_depth = 2, max_features = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)


,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.187380,0.071545,0.009075,0.074815,0.074971,0.010070,1
1,0.202681,0.070501,0.008904,0.059333,0.074517,0.010163,2
2,0.199877,0.070493,0.008937,0.026285,0.075087,0.010482,3
3,0.200881,0.070386,0.008926,0.029963,0.075073,0.010435,4
4,0.199819,0.070471,0.008938,0.028033,0.074995,0.010453,5
5,0.200761,0.070412,0.008926,0.028957,0.074960,0.010430,6
6,0.200469,0.070419,0.008931,0.029157,0.074956,0.010450,7
7,0.200935,0.070345,0.008925,0.028299,0.075039,0.010472,8


In [87]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(2, 7):
    d = my_KFold(RandomForestRegressor(max_depth = 2, max_features = 1, min_samples_split = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.187423,0.071535,0.009076,0.078405,0.074954,0.010057,2
1,0.188186,0.071453,0.009063,0.077743,0.074968,0.010063,3
2,0.186740,0.071532,0.009081,0.076741,0.075056,0.010069,4
3,0.186885,0.071502,0.009080,0.073911,0.074895,0.010075,5
4,0.186536,0.071619,0.009085,0.081379,0.074797,0.010011,6


In [88]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in [50, 100, 150, 200, 300, 400, 500]:
    d = my_KFold(RandomForestRegressor(max_depth = 2, n_estimators = i, max_features = 1), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.187872,0.071470,0.009070,0.078929,0.074735,0.010044,50
1,0.187452,0.071519,0.009075,0.081227,0.074921,0.010010,100
2,0.185883,0.071518,0.009092,0.080087,0.074973,0.010031,150
3,0.186281,0.071576,0.009089,0.080466,0.074895,0.010041,200
4,0.186322,0.071538,0.009087,0.081472,0.074816,0.010017,300
5,0.187625,0.071500,0.009072,0.078576,0.074894,0.010053,400
6,0.187191,0.071500,0.009076,0.078443,0.074954,0.010040,500


In [89]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in [50, 100, 150, 200, 300, 400, 500]:
    d = my_KFold(RandomForestRegressor(max_depth = 2, max_features = 1, n_estimators = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.181435,0.071619,0.009142,0.075514,0.075135,0.010067,50
1,0.185234,0.071593,0.009103,0.076741,0.075094,0.010066,100
2,0.187005,0.071527,0.009078,0.078022,0.075115,0.010049,150
3,0.188229,0.071451,0.009066,0.080671,0.074855,0.010036,200
4,0.185963,0.071555,0.009091,0.078186,0.074998,0.010054,300
5,0.186313,0.071589,0.009087,0.078037,0.074900,0.010050,400
6,0.186606,0.071530,0.009084,0.080221,0.074786,0.010033,500


In [91]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in [100, 150, 200, 300, 400, 500]:
    d = my_KFold(RandomForestRegressor(max_depth = 2, max_features = 1, n_estimators = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.188295,0.071484,0.009065,0.080345,0.074841,0.010047,100
1,0.186885,0.071519,0.009081,0.078788,0.075031,0.010052,150
2,0.186923,0.071536,0.009080,0.080651,0.074802,0.010023,200
3,0.186900,0.071522,0.009081,0.077149,0.075016,0.010058,300
4,0.186763,0.071529,0.009083,0.081105,0.074826,0.010019,400
5,0.186637,0.071529,0.009083,0.080530,0.074936,0.010034,500


In [93]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(100, 210, 10):
    d = my_KFold(RandomForestRegressor(max_depth = 2, max_features = 1, n_estimators = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.188424,0.071506,0.009062,0.075775,0.075067,0.010077,100
1,0.184236,0.071560,0.009110,0.077891,0.075027,0.010044,110
2,0.185879,0.071550,0.009090,0.079199,0.074879,0.010013,120
3,0.187716,0.071480,0.009069,0.075850,0.074933,0.010056,130
4,0.185930,0.071582,0.009090,0.078285,0.074947,0.010053,140
5,0.187959,0.071436,0.009068,0.072936,0.075046,0.010078,150
6,0.187224,0.071445,0.009077,0.078619,0.074995,0.010052,160
7,0.186358,0.071553,0.009086,0.075769,0.074984,0.010065,170
8,0.185851,0.071568,0.009093,0.077375,0.075061,0.010058,180
9,0.187418,0.071544,0.009074,0.076580,0.074917,0.010069,190


In [95]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(100, 210, 10):
    d = my_KFold(RandomForestRegressor(max_depth = 2, max_features = 1, n_estimators = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.185562,0.071578,0.009096,0.074637,0.075120,0.010102,100
1,0.188267,0.071451,0.009067,0.080260,0.074923,0.010038,110
2,0.185848,0.071551,0.009091,0.080994,0.074680,0.010012,120
3,0.185942,0.071475,0.009092,0.073736,0.074957,0.010095,130
4,0.189039,0.071483,0.009055,0.082273,0.074770,0.010025,140
5,0.186950,0.071496,0.009080,0.083470,0.074794,0.010010,150
6,0.185174,0.071594,0.009099,0.081573,0.074915,0.010024,160
7,0.186790,0.071586,0.009084,0.074790,0.075082,0.010080,170
8,0.186630,0.071548,0.009083,0.078304,0.075006,0.010042,180
9,0.184010,0.071650,0.009114,0.082751,0.074795,0.010023,190


In [96]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(110, 160, 2):
    d = my_KFold(RandomForestRegressor(max_depth = 2, max_features = 1, n_estimators = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.184794,0.071634,0.009105,0.076960,0.075046,0.010072,110
1,0.184618,0.071684,0.009107,0.076494,0.074876,0.010092,112
2,0.186766,0.071619,0.009083,0.082065,0.075017,0.010026,114
3,0.186793,0.071470,0.009083,0.082130,0.074728,0.010009,116
4,0.185143,0.071558,0.009100,0.080265,0.075053,0.010035,118
5,0.187026,0.071629,0.009079,0.079539,0.074738,0.010043,120
6,0.185971,0.071612,0.009092,0.079741,0.074858,0.010026,122
7,0.187499,0.071483,0.009072,0.080737,0.074753,0.010023,124
8,0.185616,0.071570,0.009097,0.080274,0.075005,0.010046,126
9,0.186159,0.071587,0.009088,0.077172,0.074897,0.010052,128


In [97]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(112, 152, 2):
    d = my_KFold(RandomForestRegressor(max_depth = 2, max_features = 1, n_estimators = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.186438,0.071602,0.009086,0.084569,0.074886,0.009985,112
1,0.185093,0.071642,0.009100,0.082395,0.074841,0.010011,114
2,0.185209,0.071582,0.009102,0.081140,0.074804,0.010019,116
3,0.187777,0.071480,0.009070,0.080765,0.074821,0.010016,118
4,0.186580,0.071530,0.009084,0.082978,0.074886,0.010013,120
5,0.188800,0.071413,0.009057,0.082939,0.074627,0.009994,122
6,0.188737,0.071377,0.009060,0.072860,0.075165,0.010120,124
7,0.187870,0.071592,0.009068,0.079345,0.074888,0.010043,126
8,0.188495,0.071437,0.009063,0.078423,0.074925,0.010036,128
9,0.186919,0.071472,0.009080,0.074900,0.075141,0.010083,130


In [99]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(112, 122):
    d = my_KFold(RandomForestRegressor(max_depth = 2, max_features = 1, n_estimators = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.186137,0.071592,0.009088,0.075070,0.074972,0.010086,112
1,0.186269,0.071556,0.009086,0.079762,0.074969,0.010022,113
2,0.186334,0.071531,0.009087,0.079236,0.075034,0.010048,114
3,0.185184,0.071646,0.009098,0.075789,0.074979,0.010088,115
4,0.186986,0.071448,0.009080,0.080275,0.074894,0.010050,116
5,0.184453,0.071661,0.009107,0.077487,0.075003,0.010043,117
6,0.188284,0.071437,0.009067,0.073796,0.075074,0.010116,118
7,0.185373,0.071619,0.009098,0.082746,0.074775,0.010020,119
8,0.187367,0.071488,0.009075,0.082547,0.074708,0.009994,120
9,0.186711,0.071502,0.009084,0.077675,0.075061,0.010067,121


In [100]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(112, 122):
    d = my_KFold(RandomForestRegressor(max_depth = 2, max_features = 1, n_estimators = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6600\1791543567.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)
C:\Users\User\AppData\Local\Temp\ipykernel_6

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.186473,0.071552,0.009085,0.077980,0.074696,0.010046,112
1,0.186180,0.071449,0.009089,0.076112,0.074883,0.010065,113
2,0.185621,0.071520,0.009096,0.082196,0.074847,0.010020,114
3,0.186150,0.071597,0.009088,0.074190,0.075036,0.010085,115
4,0.187234,0.071475,0.009075,0.079609,0.074839,0.010037,116
5,0.184445,0.071634,0.009109,0.082731,0.074898,0.010004,117
6,0.186233,0.071561,0.009088,0.082298,0.074791,0.010015,118
7,0.189639,0.071389,0.009051,0.080169,0.074850,0.010025,119
8,0.187215,0.071497,0.009077,0.083051,0.074815,0.010010,120
9,0.185141,0.071607,0.009101,0.077819,0.074924,0.010061,121


#### Valence: max_depth: 4, n_estomator = 200, max_features = 7; Arousal: max_depth: 2, n_estimator = 120, max_features = 1

# GradientBoosting

#####  Arousal

In [105]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in [0.1, 0.01, 0.001, 0.0001]:
    d = my_KFold(GradientBoostingRegressor(learning_rate = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.476285,0.055493,0.005849,-0.009012,0.073884,0.010643,0.1000
1,0.230533,0.068773,0.008603,0.070505,0.074183,0.010171,0.0100
2,0.041076,0.077376,0.010728,0.016436,0.077998,0.010983,0.0010
3,0.004444,0.078870,0.011140,-0.000023,0.078798,0.011182,0.0001


In [108]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in [None, 1, 2, 3, 4, 5]:
    d = my_KFold(GradientBoostingRegressor(max_depth = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,1.000000,0.000002,7.894892e-12,-0.841241,0.095104,0.018370,NaN
1,0.234867,0.068139,8.550143e-03,0.080108,0.073765,0.010004,1.0
2,0.361435,0.061411,7.129584e-03,0.018518,0.073554,0.010473,2.0
3,0.476285,0.055493,5.848707e-03,-0.008582,0.073886,0.010637,3.0
4,0.600410,0.048429,4.469457e-03,-0.060439,0.075005,0.011115,4.0
5,0.704273,0.041714,3.310583e-03,-0.099803,0.076172,0.011477,5.0


In [109]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(2, 8):
    d = my_KFold(GradientBoostingRegressor(min_samples_leaf = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.474438,0.055490,0.005865,-0.007787,0.073635,0.010668,2
1,0.474801,0.055434,0.005868,-0.016837,0.074168,0.010738,3
2,0.472334,0.055599,0.005893,-0.006132,0.073746,0.010660,4
3,0.466810,0.055820,0.005953,-0.012631,0.073986,0.010705,5
4,0.462898,0.056095,0.005997,-0.011144,0.074050,0.010731,6
5,0.461452,0.056147,0.006010,-0.013270,0.073955,0.010731,7


In [110]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(100, 600, 100):
    d = my_KFold(GradientBoostingRegressor(n_estimators = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.476285,0.055493,0.005849,-0.008925,0.073898,0.010641,100
1,0.589380,0.049400,0.004589,-0.057357,0.074965,0.011064,200
2,0.663517,0.044822,0.003761,-0.098094,0.076095,0.011452,300
3,0.717178,0.041215,0.003159,-0.124551,0.076939,0.011686,400
4,0.761537,0.038034,0.002665,-0.144525,0.077519,0.011867,500


In [112]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(2, 9):
    d = my_KFold(GradientBoostingRegressor(min_samples_split  = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.476285,0.055493,0.005849,-0.008870,0.073903,0.010640,2
1,0.478187,0.055404,0.005829,-0.015979,0.073923,0.010716,3
2,0.478430,0.055399,0.005826,-0.009850,0.073927,0.010669,4
3,0.477610,0.055398,0.005834,-0.015761,0.074016,0.010721,5
4,0.475721,0.055549,0.005857,-0.014262,0.073939,0.010698,6
5,0.476109,0.055570,0.005850,-0.011749,0.074010,0.010688,7
6,0.475292,0.055558,0.005858,-0.010034,0.073957,0.010667,8


In [113]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(1, 9):
    d = my_KFold(GradientBoostingRegressor(max_features  = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.443102,0.057344,0.006221,0.012569,0.073343,0.010489,1
1,0.468303,0.056006,0.005943,0.001433,0.073669,0.010607,2
2,0.476285,0.055493,0.005849,-0.008940,0.073890,0.010642,3
3,0.476285,0.055493,0.005849,-0.009072,0.073895,0.010641,4
4,0.476285,0.055493,0.005849,-0.009353,0.073897,0.010647,5
5,0.476285,0.055493,0.005849,-0.008914,0.073880,0.010640,6
6,0.476285,0.055493,0.005849,-0.009095,0.073903,0.010643,7
7,0.476285,0.055493,0.005849,-0.008433,0.073867,0.010637,8


In [120]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(5):
    d = my_KFold(GradientBoostingRegressor(min_impurity_decrease = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.476285,0.055493,0.005849,-0.009298,0.073902,0.010643,0
1,0.098565,0.074860,0.010061,0.033485,0.077089,0.010738,1
2,0.041905,0.077306,0.010698,0.009886,0.078343,0.011069,2
3,0.010547,0.078621,0.011062,0.000524,0.078748,0.011187,3
4,0.000000,0.079047,0.011190,-0.002118,0.078890,0.011206,4


In [121]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in [0.1, 0.01, 0.001, 0.0001]:
    d = my_KFold(GradientBoostingRegressor(learning_rate = i, max_depth = 1), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.234867,0.068139,0.008550,0.080108,0.073765,0.010004,0.1000
1,0.119074,0.074096,0.009844,0.052056,0.076135,0.010496,0.0100
2,0.021303,0.078212,0.010950,0.009564,0.078408,0.011073,0.0010
3,0.002307,0.078958,0.011164,-0.000818,0.078839,0.011191,0.0001


In [122]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(5):
    d = my_KFold(GradientBoostingRegressor(min_impurity_decrease = i, max_depth = 1), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.234867,0.068139,0.008550,0.080108,0.073765,0.010004,0
1,0.097201,0.074923,0.010078,0.034258,0.077041,0.010732,1
2,0.041905,0.077306,0.010698,0.009886,0.078343,0.011069,2
3,0.010547,0.078621,0.011062,0.000524,0.078748,0.011187,3
4,0.000000,0.079047,0.011190,-0.002118,0.078890,0.011206,4


In [124]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(1, 9):
    d = my_KFold(GradientBoostingRegressor(max_features = i, max_depth = 1), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.232164,0.068375,0.00858,0.081701,0.073667,0.009988,1
1,0.234863,0.068142,0.00855,0.080987,0.073708,0.009996,2
2,0.234867,0.068139,0.00855,0.080108,0.073765,0.010004,3
3,0.234867,0.068139,0.00855,0.080108,0.073765,0.010004,4
4,0.234867,0.068139,0.00855,0.080108,0.073765,0.010004,5
5,0.234867,0.068139,0.00855,0.080108,0.073765,0.010004,6
6,0.234867,0.068139,0.00855,0.080108,0.073765,0.010004,7
7,0.234867,0.068139,0.00855,0.080108,0.073765,0.010004,8


In [129]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(1, 13):
    d = my_KFold(GradientBoostingRegressor(max_features = 1, max_depth = 1, min_samples_leaf = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.233057,0.068359,0.008569,0.080663,0.073718,0.009988,1
1,0.233053,0.068295,0.008571,0.081986,0.073745,0.009978,2
2,0.232328,0.068302,0.008578,0.080474,0.073716,0.009986,3
3,0.232790,0.068329,0.008574,0.080659,0.073693,0.009979,4
4,0.232761,0.068345,0.008573,0.081033,0.073714,0.009987,5
5,0.233344,0.068288,0.008567,0.081209,0.073772,0.009984,6
6,0.232371,0.068407,0.008578,0.083344,0.073640,0.009966,7
7,0.233607,0.068262,0.008563,0.082524,0.073588,0.009959,8
8,0.233482,0.068271,0.008565,0.082877,0.073700,0.009965,9
9,0.233250,0.068293,0.008568,0.083698,0.073573,0.009941,10


In [130]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(1, 13):
    d = my_KFold(GradientBoostingRegressor(max_features = 1, max_depth = 1, min_samples_leaf = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.233220,0.068267,0.008568,0.081018,0.073609,0.009968,1
1,0.233284,0.068342,0.008567,0.082579,0.073843,0.009964,2
2,0.233090,0.068260,0.008569,0.078445,0.073942,0.010029,3
3,0.233050,0.068267,0.008571,0.076032,0.073787,0.010016,4
4,0.233049,0.068302,0.008570,0.078375,0.073758,0.010005,5
5,0.233504,0.068283,0.008565,0.081494,0.073744,0.009982,6
6,0.233075,0.068321,0.008569,0.086838,0.073461,0.009913,7
7,0.233793,0.068234,0.008561,0.082746,0.073770,0.009992,8
8,0.233444,0.068299,0.008566,0.080072,0.073895,0.010006,9
9,0.232011,0.068393,0.008582,0.078820,0.073819,0.010009,10


In [131]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(1, 13):
    d = my_KFold(GradientBoostingRegressor(max_features = 1, max_depth = 1, min_samples_leaf = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.233543,0.068291,0.008565,0.081683,0.073599,0.009978,1
1,0.231610,0.068455,0.008585,0.081303,0.073572,0.009967,2
2,0.233851,0.068232,0.008561,0.078190,0.073883,0.010024,3
3,0.233398,0.068290,0.008567,0.077753,0.073824,0.010010,4
4,0.233401,0.068228,0.008566,0.083226,0.073685,0.009964,5
5,0.232273,0.068376,0.008579,0.080607,0.073732,0.009976,6
6,0.233193,0.068231,0.008568,0.079696,0.073667,0.009991,7
7,0.233726,0.068232,0.008563,0.080720,0.073784,0.009989,8
8,0.233086,0.068306,0.008570,0.081414,0.073704,0.009987,9
9,0.233310,0.068273,0.008567,0.080614,0.073592,0.009976,10


In [133]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(1, 13):
    d = my_KFold(GradientBoostingRegressor(max_features = 1, max_depth = 1, min_samples_leaf = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.231272,0.068439,0.008589,0.080782,0.073692,0.009972,1
1,0.234040,0.068239,0.008559,0.083105,0.073655,0.009972,2
2,0.233807,0.068199,0.008562,0.081151,0.073776,0.009985,3
3,0.233015,0.068338,0.008571,0.079042,0.073860,0.009994,4
4,0.233717,0.068282,0.008563,0.079659,0.073797,0.010006,5
5,0.232214,0.068392,0.008579,0.083870,0.073677,0.009944,6
6,0.233306,0.068299,0.008567,0.081208,0.073599,0.009976,7
7,0.233669,0.068253,0.008564,0.081187,0.073736,0.009987,8
8,0.232849,0.068293,0.008572,0.081204,0.073677,0.009964,9
9,0.233417,0.068280,0.008567,0.080431,0.073775,0.009996,10


In [141]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in [10, 50, 100, 150, 200, 300, 400, 500]:
    d = my_KFold(GradientBoostingRegressor(max_features = 1, max_depth = 1, n_estimators = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.112373,0.074645,0.009918,0.066307,0.076134,0.010371,10
1,0.207328,0.069969,0.008854,0.084141,0.074149,0.010005,50
2,0.232925,0.068273,0.008571,0.081188,0.073629,0.009983,100
3,0.243807,0.067522,0.008452,0.074205,0.073694,0.010032,150
4,0.250275,0.067078,0.008380,0.074097,0.073640,0.010041,200
5,0.259647,0.066527,0.008276,0.072277,0.073558,0.010051,300
6,0.265407,0.066201,0.008212,0.065995,0.073721,0.010105,400
7,0.269953,0.065940,0.008162,0.063086,0.073832,0.010135,500


In [142]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(90, 210, 10):
    d = my_KFold(GradientBoostingRegressor(max_features = 1, max_depth = 1, n_estimators = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.229987,0.068508,0.008605,0.078616,0.073875,0.010012,90
1,0.233647,0.068284,0.008562,0.083459,0.073589,0.009946,100
2,0.235818,0.068115,0.008540,0.084742,0.073702,0.009953,110
3,0.238067,0.067916,0.008515,0.079385,0.073676,0.009984,120
4,0.240355,0.067740,0.008490,0.080427,0.073588,0.009988,130
5,0.242034,0.067661,0.008472,0.074791,0.073770,0.010031,140
6,0.244203,0.067493,0.008447,0.077954,0.073589,0.009996,150
7,0.245093,0.067410,0.008437,0.076951,0.073545,0.010011,160
8,0.246027,0.067384,0.008428,0.077647,0.073559,0.010005,170
9,0.248196,0.067255,0.008403,0.076602,0.073625,0.010010,180


In [144]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(90, 135, 5):
    d = my_KFold(GradientBoostingRegressor(max_features = 1, max_depth = 1, n_estimators = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.229937,0.068507,0.008604,0.081342,0.073702,0.009968,90
1,0.231569,0.068431,0.008587,0.083659,0.073708,0.009967,95
2,0.232917,0.068329,0.008571,0.078232,0.073796,0.010009,100
3,0.235584,0.068125,0.008542,0.081063,0.073719,0.009987,105
4,0.235311,0.068151,0.008545,0.081532,0.073628,0.009973,110
5,0.236406,0.068048,0.008533,0.076722,0.073900,0.010037,115
6,0.239172,0.067822,0.008502,0.076636,0.073751,0.010031,120
7,0.240245,0.067813,0.008491,0.079669,0.073608,0.009979,125
8,0.240465,0.067779,0.008487,0.081293,0.073386,0.009954,130


In [145]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(90, 101):
    d = my_KFold(GradientBoostingRegressor(max_features = 1, max_depth = 1, n_estimators = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.229758,0.068513,0.008606,0.084473,0.073596,0.009933,90
1,0.231153,0.068380,0.008590,0.084486,0.073631,0.009953,91
2,0.230873,0.068416,0.008594,0.085711,0.073644,0.009933,92
3,0.231009,0.068418,0.008592,0.079842,0.073841,0.009990,93
4,0.230608,0.068478,0.008596,0.080639,0.073627,0.009980,94
5,0.231345,0.068432,0.008588,0.086543,0.073510,0.009917,95
6,0.232303,0.068389,0.008578,0.082125,0.073782,0.009971,96
7,0.232571,0.068373,0.008575,0.084608,0.073729,0.009949,97
8,0.232003,0.068410,0.008583,0.081804,0.073687,0.009974,98
9,0.233396,0.068294,0.008566,0.085453,0.073584,0.009946,99


In [148]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(2, 13):
    d = my_KFold(GradientBoostingRegressor(max_features = 1, max_depth = 1, n_estimators = 95, min_samples_split = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.230889,0.068482,0.008594,0.081150,0.073840,0.009993,2
1,0.231392,0.068446,0.008590,0.077158,0.073852,0.010007,3
2,0.231838,0.068420,0.008584,0.079730,0.073812,0.009993,4
3,0.231184,0.068417,0.008590,0.080544,0.073716,0.009988,5
4,0.231301,0.068454,0.008588,0.083691,0.073614,0.009954,6
5,0.232226,0.068315,0.008579,0.082515,0.073706,0.009975,7
6,0.232016,0.068342,0.008582,0.084749,0.073657,0.009964,8
7,0.231690,0.068422,0.008585,0.081455,0.073684,0.009971,9
8,0.231609,0.068401,0.008585,0.083478,0.073666,0.009965,10
9,0.231467,0.068444,0.008587,0.081253,0.073886,0.010009,11


## Valence

In [161]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(1, 8):
    d = my_KFold(GradientBoostingRegressor(max_depth = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.363454,0.231633,0.089774,0.246542,0.246215,0.100086,1
1,0.465336,0.210311,0.075450,0.279877,0.239076,0.095923,2
2,0.558851,0.190181,0.062327,0.266519,0.240000,0.097753,3
3,0.648199,0.169666,0.049691,0.258642,0.238668,0.097787,4
4,0.735825,0.146873,0.037319,0.223677,0.243943,0.101670,5
5,0.804724,0.124818,0.027539,0.190093,0.246834,0.106008,6
6,0.874400,0.099166,0.017708,0.137465,0.253243,0.112210,7


In [162]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in [1, 0.1, 0.001, 0.0001]:
    d = my_KFold(GradientBoostingRegressor(max_depth = 2, learning_rate = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.697908,0.154730,0.042646,-0.023724,0.275609,0.131600,1.0000
1,0.465336,0.210311,0.075450,0.279742,0.239109,0.095942,0.1000
2,0.051689,0.274165,0.133797,0.017554,0.275118,0.134314,0.0010
3,0.005624,0.278622,0.140298,-0.023599,0.279094,0.140262,0.0001


In [163]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(5):
    d = my_KFold(GradientBoostingRegressor(max_depth = 2, min_impurity_decrease = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.465336,0.210311,0.075450,0.279598,0.239128,0.095965,0
1,0.370746,0.231206,0.088706,0.258497,0.246487,0.099522,1
2,0.344564,0.236355,0.092424,0.248974,0.247394,0.100783,2
3,0.324034,0.239897,0.095276,0.239010,0.248978,0.102308,3
4,0.311129,0.242143,0.097082,0.233463,0.249964,0.103209,4


In [164]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(1, 9):
    d = my_KFold(GradientBoostingRegressor(max_depth = 2, max_features = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.453009,0.213709,0.077218,0.273653,0.242255,0.097015,1
1,0.464567,0.210768,0.075559,0.288651,0.238270,0.094908,2
2,0.465336,0.210311,0.075450,0.279547,0.239152,0.095966,3
3,0.465336,0.210311,0.075450,0.279597,0.239138,0.095957,4
4,0.465336,0.210311,0.075450,0.279552,0.239143,0.095972,5
5,0.465336,0.210311,0.075450,0.279713,0.239112,0.095947,6
6,0.465336,0.210311,0.075450,0.279804,0.239087,0.095938,7
7,0.465336,0.210311,0.075450,0.279619,0.239140,0.095958,8


In [166]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(2, 9):
    d = my_KFold(GradientBoostingRegressor(max_depth = 2, max_features = 2, min_samples_split = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.466008,0.210983,0.075375,0.288546,0.238750,0.094608,2
1,0.465761,0.211229,0.075410,0.283460,0.239373,0.095289,3
2,0.465894,0.210699,0.075393,0.282844,0.239192,0.095428,4
3,0.465021,0.211184,0.075481,0.280472,0.239928,0.096029,5
4,0.463916,0.210778,0.075661,0.286470,0.238501,0.095062,6
5,0.463976,0.210787,0.075632,0.280125,0.240214,0.095626,7
6,0.463251,0.211313,0.075706,0.286561,0.238981,0.094931,8


In [168]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(1, 9):
    d = my_KFold(GradientBoostingRegressor(max_depth = 2, max_features = 2, min_samples_leaf = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.467047,0.210949,0.075211,0.285190,0.239437,0.095144,1
1,0.464086,0.210585,0.075603,0.281829,0.239414,0.095486,2
2,0.464873,0.210977,0.075535,0.283738,0.239347,0.095380,3
3,0.467623,0.210315,0.075138,0.280019,0.240305,0.095880,4
4,0.464212,0.210966,0.075610,0.284911,0.239714,0.095156,5
5,0.468239,0.210180,0.075077,0.287557,0.238552,0.094990,6
6,0.468066,0.210186,0.075083,0.283310,0.239216,0.095344,7
7,0.468348,0.210481,0.075049,0.274278,0.241045,0.096346,8


In [170]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(100, 600, 100):
    d = my_KFold(GradientBoostingRegressor(max_depth = 2, max_features = 2, n_estimators = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.463443,0.211276,0.075741,0.284222,0.239592,0.095198,100
1,0.523074,0.198190,0.067358,0.268220,0.240078,0.097320,200
2,0.563737,0.189212,0.061624,0.254773,0.241095,0.098630,300
3,0.595898,0.181853,0.057061,0.240625,0.242516,0.099720,400
4,0.621123,0.175999,0.053496,0.220103,0.245606,0.102199,500


In [171]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(100, 700, 100):
    d = my_KFold(GradientBoostingRegressor(max_depth = 2, max_features = 2, n_estimators = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.465612,0.210418,0.075414,0.276018,0.241187,0.096302,100
1,0.522087,0.198164,0.067463,0.258594,0.240204,0.098311,200
2,0.565267,0.189292,0.061403,0.249516,0.241779,0.098940,300
3,0.594766,0.182394,0.057244,0.243336,0.242249,0.099676,400
4,0.622056,0.175576,0.053358,0.225582,0.244597,0.101163,500
5,0.644390,0.170526,0.050209,0.219361,0.245773,0.102076,600


In [172]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(100, 600, 100):
    d = my_KFold(GradientBoostingRegressor(max_depth = 2, max_features = 2, n_estimators = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.468083,0.210141,0.075059,0.279483,0.240497,0.096098,100
1,0.526164,0.197669,0.066923,0.259803,0.240668,0.098076,200
2,0.564207,0.189007,0.061535,0.253333,0.240145,0.098547,300
3,0.596410,0.181593,0.056974,0.237366,0.242259,0.100199,400
4,0.622809,0.175726,0.053250,0.221131,0.244211,0.101799,500


# SVR

In [175]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(1, 5):
    d = my_KFold(SVR(kernel = 'poly', degree = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.282728,0.241135,0.101231,0.243664,0.243445,0.102035,1
1,0.213506,0.246991,0.111042,0.115739,0.258324,0.119734,2
2,0.197036,0.250231,0.113492,-0.032504,0.268233,0.147442,3
3,0.158464,0.255098,0.118890,0.000310,0.269015,0.139472,4


In [174]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in ['linear', 'poly', 'rbf', 'sigmoid']:
    d = my_KFold(SVR(kernel = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.282672,0.241133,0.101239,0.243146,0.243479,0.102071,linear
1,0.197036,0.250231,0.113492,-0.032504,0.268233,0.147442,poly
2,0.345913,0.222690,0.092345,0.239915,0.244980,0.104023,rbf
3,-11831.078460,30.413985,1672.540938,-10352.112509,28.370345,1371.216689,sigmoid


In [178]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in np.arange(0.1, 2.1, 0.1):
    d = my_KFold(SVR(kernel = 'linear', C = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.282794,0.241132,0.101221,0.243459,0.243580,0.102087,0.1
1,0.282742,0.241132,0.101228,0.243564,0.243526,0.102072,0.2
2,0.282812,0.241138,0.101219,0.243511,0.243517,0.102057,0.3
3,0.282781,0.241135,0.101224,0.243417,0.243520,0.102064,0.4
4,0.282656,0.241126,0.101243,0.242935,0.243536,0.102097,0.5
5,0.282664,0.241129,0.101240,0.242950,0.243509,0.102094,0.6
6,0.282643,0.241126,0.101241,0.243002,0.243474,0.102088,0.7
7,0.282734,0.241136,0.101230,0.242922,0.243502,0.102092,0.8
8,0.282691,0.241129,0.101236,0.243008,0.243504,0.102087,0.9
9,0.282672,0.241133,0.101239,0.243146,0.243479,0.102071,1.0


In [180]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in ['scale', 'auto']:
    d = my_KFold(SVR(kernel = 'linear', gamma = i), 5, data_train, 'Valence')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.282672,0.241133,0.101239,0.243146,0.243479,0.102071,scale
1,0.282672,0.241133,0.101239,0.243146,0.243479,0.102071,auto


In [181]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in ['linear', 'poly', 'rbf', 'sigmoid']:
    d = my_KFold(SVR(kernel = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.093826,0.077736,0.010141,0.054374,0.078649,0.010462,linear
1,0.096801,0.076830,0.010118,-0.193529,0.080800,0.012440,poly
2,0.208697,0.074014,0.008836,-0.041661,0.080816,0.011434,rbf
3,-166303.062431,31.217494,1846.020552,-205069.025784,30.931865,1806.148090,sigmoid


In [183]:
d_nw = {'r2_train':[],
        'mae_train':[],
        'mse_train':[],
         'r2_test':[],
         'mae_test':[],
         'mse_test':[],
        'params':[]}
for i in range(1, 5):
    d = my_KFold(SVR(kernel = 'poly', degree = i), 5, data_train, 'Arousal')
    d_nw['r2_train'].append(d['r2_train'])
    d_nw['r2_test'].append(d['r2_test'])
    d_nw['mae_train'].append(d['mae_train'])
    d_nw['mae_test'].append(d['mae_test'])
    d_nw['mse_train'].append(d['mse_train'])
    d_nw['mse_test'].append(d['mse_test'])
    d_nw['params'].append(i)
pd.DataFrame(d_nw)

C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\miniconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

,r2_train,mae_train,mse_train,r2_test,mae_test,mse_test,params
0,0.094550,0.077649,0.010134,0.055206,0.078543,0.010450,1
1,0.069897,0.078128,0.010404,-0.035314,0.079596,0.011269,2
2,0.096801,0.076830,0.010118,-0.193529,0.080800,0.012440,3
3,0.087033,0.077287,0.010225,-0.495389,0.082982,0.014689,4
